<a href="https://colab.research.google.com/github/unt-iialab/medical-concept-normalization/blob/master/medical-concept-normalization/fine-tuning-experiments/askpatient/ask_4_mixed_pubmed_bigtweet_ask.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os

In [2]:
base_dir = 'medical-concept-normalization'

In [3]:
os.mkdir(base_dir)

In [4]:
os.mkdir('medical-concept-normalization/data_collection')

**Install fastai and clone the fastai for text classification**



In [5]:
!curl https://course-v3.fast.ai/setup/colab | bash

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0curl: (6) Could not resolve host: course-v3.fast.ai


In [6]:
!git clone https://github.com/wshuyi/demo-nlp-classification-fastai.git

Cloning into 'demo-nlp-classification-fastai'...
remote: Enumerating objects: 9, done.
remote: Total 9 (delta 0), reused 0 (delta 0), pack-reused 9
Unpacking objects: 100% (9/9), done.


In [7]:
from fastai import *
from fastai.text import *

**Set the path of the data collections**

In [8]:
path = Path('medical-concept-normalization/data_collection')

In [10]:
# training data set

train_csv = path/'AskAPatient.fold-4.train.csv'
train = pd.read_csv(train_csv, header=None)
#train.head()
#train.info()

In [11]:
# validating data set

valid = pd.read_csv(path/"AskAPatient.fold-4.validation.csv",header=None)
valid.head()

,0,1
0,6,diastolic dysfunction
1,8,ARTHRITIS
2,8,arthritis
3,8,arthritis
4,8,arthritis


In [12]:
# testing data set

test = pd.read_csv(path/"AskAPatient.fold-4.test.csv",header=None)
test.head()

,0,1
0,4,purple sploches around waist
1,6,stage 1 diastolic dysfunction
2,8,arthritis
3,12,shingles
4,16,flu


In [13]:
# Creating DataBunch for language modelling

data_lm = TextLMDataBunch.from_csv(path, 'mixed_pubmed_bigtweet_askpatient.csv')

In [14]:
# Creating DataBunch for classification

# data_clas = TextClasDataBunch.from_df(path, train, valid, test, vocab=data_lm.train_ds.vocab, bs=128)

In [15]:
data_clas = TextClasDataBunch.from_df(path, train, valid, vocab=data_lm.train_ds.vocab, bs=128)

In [16]:
data_clas.add_test(test,label=0)


In [17]:
data_lm.save()
data_clas.save()

In [18]:
data_lm.train_ds.vocab.itos

['xxunk',
 'xxpad',
 'xxbos',
 'xxeos',
 'xxfld',
 'xxmaj',
 'xxup',
 'xxrep',
 'xxwrep',
 'pain',
 'of',
 'the',
 'in',
 'lipitor',
 'and',
 'to',
 'a',
 'muscle',
 'with',
 'for',
 'was',
 'diclofenac',
 'were',
 'severe',
 'on',
 'my',
 'is',
 'myalgia',
 'lower',
 'rt',
 'fatigue',
 'loss',
 'by',
 'cramp',
 'limb',
 'sodium',
 'weakness',
 'at',
 'disease',
 'stomach',
 'that',
 'or',
 'as',
 'back',
 'depression',
 '-',
 'from',
 'patients',
 'i',
 'not',
 'this',
 'cramps',
 'upper',
 'gas',
 'joint',
 'drug',
 'memory',
 'mg',
 'leg',
 'you',
 'all',
 'pains',
 'legs',
 'headache',
 'are',
 'knee',
 'be',
 'group',
 'shoulder',
 'unable',
 'study',
 'gastrointestinal',
 'arthralgia',
 'foot',
 'neck',
 'excessive',
 'treatment',
 'after',
 'arthritis',
 'aches',
 'it',
 'an',
 'amp',
 'hip',
 'have',
 'voltaren',
 'like',
 'numbness',
 'insomnia',
 'p',
 'increased',
 'arthrotec',
 'release',
 '/',
 'bleeding',
 'no',
 'time',
 'nausea',
 'both',
 'can',
 'abdominal',
 'gluten'

In [19]:
model_path = path/'models'
model_path.mkdir(exist_ok=True)
url = 'http://files.fast.ai/models/wt103_v1/'
download_url(f'{url}lstm_wt103.pth', model_path/'lstm_wt103.pth')
download_url(f'{url}itos_wt103.pkl', model_path/'itos_wt103.pkl')

In [20]:
learn = language_model_learner(data_lm, AWD_LSTM, drop_mult=0.5)

In [21]:
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.856901,3.486236,0.441621,32:59


In [22]:
# learn.fit_one_cycle(1, 1e-2)

In [23]:
# learn.fit_one_cycle(1, 1e-3)

In [24]:
learn.fit_one_cycle(1, 1e-3)

epoch,train_loss,valid_loss,accuracy,time
0,3.517488,3.424687,0.447500,33:01


In [25]:
learn.save_encoder('pubmed_cadec_askpatient_mixed_fold4_first')

In [26]:
learn = text_classifier_learner(data_clas,AWD_LSTM, drop_mult=0.5)
learn.load_encoder('pubmed_cadec_askpatient_mixed_fold4_first')
learn.fit_one_cycle(1, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,3.842343,3.302469,0.414251,01:03


In [27]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(5e-3/2., 5e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.813200,2.595326,0.553140,01:15


In [28]:
learn.unfreeze()
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,2.138263,2.320685,0.599034,03:02


In [29]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.906707,1.943305,0.658213,03:06
1,1.599997,1.860266,0.669082,03:05


In [30]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.533355,1.689554,0.695652,03:04
1,1.280653,1.611044,0.708937,03:03


In [31]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.202923,1.472961,0.735507,03:16
1,0.994246,1.428030,0.752415,02:56


In [32]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.967466,1.369295,0.745169,03:18
1,0.822854,1.316201,0.754831,03:10


In [33]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.774477,1.274524,0.771739,03:07
1,0.701650,1.235138,0.783816,03:15


In [34]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.662734,1.225501,0.782609,03:05
1,0.682553,1.169761,0.797101,03:11
2,0.518776,1.134026,0.803140,02:56
3,0.448441,1.114792,0.806763,03:00


In [35]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.478928,1.149776,0.801932,03:16
1,0.485435,1.118189,0.810386,03:10
2,0.400410,1.091132,0.809179,03:20
3,0.369249,1.101190,0.817633,02:57


In [36]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.367550,1.094384,0.810386,02:58
1,0.387564,1.114738,0.817633,03:01
2,0.346811,1.114404,0.826087,03:20
3,0.295157,1.079979,0.821256,03:12


In [37]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.298226,1.113958,0.828502,03:13
1,0.322803,1.195982,0.816425,03:09
2,0.294613,1.140879,0.821256,03:09
3,0.250965,1.094577,0.822464,03:18


In [38]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.245714,1.173129,0.828502,03:07
1,0.293208,1.201228,0.829710,03:12
2,0.270362,1.153696,0.822464,03:06
3,0.218029,1.164735,0.822464,03:04


In [39]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.217214,1.187635,0.820048,03:15
1,0.248647,1.194070,0.817633,03:11
2,0.224169,1.156168,0.821256,03:15
3,0.195891,1.176072,0.827295,03:12


In [40]:
learn.fit_one_cycle(4, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.208677,1.218380,0.826087,03:10
1,0.241206,1.182953,0.820048,02:59
2,0.205548,1.177328,0.830918,02:55
3,0.180860,1.229597,0.832126,03:07


In [41]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.218836,1.237530,0.824879,03:11
1,0.193763,1.221909,0.830918,03:02


In [42]:
learn.fit_one_cycle(2, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.203004,1.235598,0.829710,03:07
1,0.189600,1.275711,0.836957,03:12


In [43]:
learn.fit_one_cycle(1, slice(2e-3/100, 2e-3))

epoch,train_loss,valid_loss,accuracy,time
0,0.189103,1.246416,0.840580,03:09


In [44]:
# save the best model

learn.save_encoder('pubmed_cadec_askpatient_mixed_fold4')

# Part three: Predict on the test dataset

In [45]:
# use the test data for prediction

preds, y = learn.get_preds(DatasetType.Test)

In [46]:
predictions, *_ = learn.get_preds(DatasetType.Test)
labels = np.argmax(predictions, 1)
predict_list = labels.tolist()
standard_list = test[0].tolist()

print(len(predict_list))
print(len(standard_list))

print(predict_list)
print(standard_list)

866
866
[862, 1035, 8, 12, 16, 359, 27, 27, 27, 29, 31, 31, 31, 31, 31, 32, 37, 37, 37, 37, 37, 37, 37, 42, 48, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 49, 53, 53, 53, 853, 226, 60, 61, 61, 62, 64, 76, 76, 76, 614, 289, 614, 1026, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 289, 614, 79, 80, 80, 80, 268, 91, 91, 92, 92, 92, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 94, 75, 94, 94, 475, 361, 96, 100, 265, 226, 650, 108, 108, 125, 289, 289, 289, 111, 111, 111, 289, 113, 116, 615, 120, 122, 122, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 125, 129, 131, 139, 140, 143, 147, 157, 160, 882, 882, 166, 166, 101, 172, 172, 172, 172, 172, 172, 172, 76, 178, 182, 182, 182, 182, 182, 182, 182, 698, 183, 183, 183, 183, 183, 183, 183, 183, 183, 183, 186, 187, 187, 188, 189, 189, 463, 190, 190, 190, 190, 190, 606, 289, 34, 197, 205, 206, 207, 208, 210, 214, 299, 214, 214, 214, 214

In [47]:
correct_num = 0

for i in range(0, len(standard_list)):
  if predict_list[i] == standard_list[i]:
    correct_num = correct_num+1
  else:
    pass
print(correct_num)

predict_accuracy = correct_num/len(predict_list)
print(predict_accuracy)

641
0.7401847575057737
